In [8]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANG_SMITH")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [9]:
# PDF 파일 로드

from langchain_community.document_loaders import PyPDFLoader

file_path = r"c:\Users\kmidd\Desktop\2406.15057v1.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [10]:
# 지식 베이스 생성

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":6})

In [12]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")

# RAG chain 만들기

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input":"What is latent space?"})

results

{'input': 'What is latent space?',
 'context': [Document(metadata={'page': 11, 'source': 'c:\\Users\\kmidd\\Desktop\\2406.15057v1.pdf'}, page_content='Latent Space Translation via Inverse Relative Projection\n2020, Addis Ababa, Ethiopia, April 26-30, 2020 . OpenRe-\nview.net, 2020. URL https://openreview.net/\nforum?id=HyebplHYwB .\nVedula, S., Maiorca, V ., Basile, L., Locatello, F., and Bron-\nstein, A. Scalable unsupervised alignment of general\nmetric and non-metric structures, 2024.\nVuli´c, I., Ruder, S., and Søgaard, A. Are all good word\nvector spaces isomorphic? In Proceedings of the\n2020 Conference on Empirical Methods in Natural Lan-\nguage Processing (EMNLP) , pp. 3178–3192, Online,\n2020. Association for Computational Linguistics. doi:\n10.18653/v1/2020.emnlp-main.257. URL https://\naclanthology.org/2020.emnlp-main.257 .\nWang, Y ., Liu, Y ., and Ma, Z.-M. The scale-invariant space\nfor attention layer in neural network. Neurocomputing ,\n392:1–10, 2020.\nWang, Z., Shan, 